In [1]:
import pandas as pd
import numpy as np

### trades offs

strategies to indicate missing values:

- masking: an extra boolean array indicating missing values
    or one byte of extra memory per value
- sentinel: a specific value indicating missing values, could be data type specific, or
    a global value like NaN

pros and cons considerations: storage conputation costs, reduction of range of values, 
    extra logic to handle missing values, availability of data type specific sentinel values.

pandas uses sentinel values:

- NaN for float
- NaT for datetime
- None for object
- in future, pandas will use Int64NA for integer

### none: python missing data

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ["object", "int"]:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
61.5 ms ± 3.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.8 ms ± 39.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)



In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### nan: numerical missing data

In [5]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [6]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### pandas conversion and sentinal value choices

In [7]:
pd.Series([1, np.nan, 2, None])  # convert between nan and None

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [8]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [9]:
x[0] = None
x  # upcast to float, since None is not a valid integer

0    NaN
1    1.0
dtype: float64

### ops - detecting

In [11]:
data = pd.Series([1, np.nan, "hello", None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [12]:
data[data.notnull()]

0        1
2    hello
dtype: object

### ops - dropping

In [13]:
data.dropna()

0        1
2    hello
dtype: object

In [14]:
df = pd.DataFrame([[1, np.nan, 2], [2, 3, 5], [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [15]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [16]:
df.dropna(axis="columns")

,2
0,2
1,5
2,6


In [17]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [18]:
df.dropna(axis="columns", how="all")

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [19]:
df.dropna(axis="rows", thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


### ops - fill

In [20]:
data = pd.Series([1, np.nan, 2, None, 3], index=list("abcde"))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [21]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [23]:
data.ffill()

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [24]:
data.bfill()

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [25]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [26]:
df.ffill(axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
